# Predicción de Utilidades de Películas

# Predicción de Utilidades de Películas

- Conjunto de entrada X
    - Dataset dispersos
        - Requiere de estructuras de datos especiales (compresión y algoritmos)
        - Formato csc_matrix (*Compressed Sparse Column matrix*)
    - Entrada con 145256 características
    - Conjunto de entrenamiento: 1147 ejemplos
    - Conjunto de validación: 317 ejemplos
    - Conjunto de pruebas: 254 ejemplos

## Regresión Lineal

Meta inicial: Regresión Lineal Ordinaria

- $R^2_{train} = 1$
- $R^2_{val} = 0.61$

- $R^2_{test} = 0.60$

```python
import sklearn.linear_model as lm
linreg = lm.LinearRegression(fit_intercept=False)
linreg.fit(X,y)
print "training R2=%f"%linreg.score(X,y)
print "validate R2=%f"%linreg.score(Xv,yv)
print "test R2=%f"%linreg.score(Xt,yt)
```

## Enfoques usados

Se probaron distintos enfoques. Los más destacables siendo:

- Umbral de varianza
- Forward Stepwise Selection
- Regresión Lasso

No obstante ningún enfoque logro una mejora substancial sobre la regresión lineal anteriormente vista e incluso algunos se podrían considerar invali.

## Umbral de Varianza

Umbral de Varianza (`sklearn.feature_selection.VarianceThreshold`) es un selector básico de características

Elimina aquellas características con baja varianza

Se probó con umbrales de varianza de: $0.5$, $0.1$, $0.01$ y $0.001$

## Umbral de Varianza

```python
selector = fs.VarianceThreshold(threshold=0.5)
Xn = selector.fit_transform(X)
Xnt = selector.transform(Xt); Xnv = selector.transform(Xv)
linreg.fit(Xn,y)
print Xn.shape
print "training R2=%f"%linreg.score(Xn,y)
print "validate R2=%f"%linreg.score(Xnv,yv)
print "test R2=%f"%linreg.score(Xnt,yt)
```



## Umbral de Varianza


```
(1147, 1087)
training R2=0.991065
validate R2=-1.346798
test R2=-0.464357
(1147, 6214)
training R2=1.000000
validate R2=0.567887
test R2=0.553110
(1147, 58685)
training R2=1.000000
validate R2=0.611514
test R2=0.593959
(1147, 145222)
training R2=1.000000
validate R2=0.612862
test R2=0.590313

```

## Forward Stepwise Selection

Se aplico Forward Stepwise Selection

En cada iteración se revisaban solo 10 características seleccionadas al azar

1. Coeficientes del modelo
2. $R^2$ sobre el conjunto de validación
3. Balance de $R^2$ sobre los conjuntos de entrenamiento y validación

### FSS: Coeficientes del Modelo

Se hizo uso de coeficientes del modelo como medida de puntaje

```python
#indexes los indices de las características ingresadas al modelo en esta iteración más el candidato
score = abs(model.coef_[indexes.index(candidate)])
```

Fuerte sobreajuste, $R^2_{val}, R^2_{test} < 0$
```
training R2=0.999654
validate R2=-0.965488
test R2=-0.658075
```

Otros enfoques de FSS arrojaron resultados similares o peores


## Regresión Lasso

Se probó regresión Lasso para ver si eliminaba correctamente las características poco importantes del modelo.

Se hizo una búsqueda semi-manual del parámetro $\alpha$ que maximizara $R^2$ sobre el conjunto de pruebas. 

La búsqueda se realizó en el rango $[10^{100},10^0]$



## Regresión Lasso
```
#alphas_ = np.logspace(100,10,base=10)
#alphas_ = np.logspace(10,0,base=10,num=10)
#alphas_ = np.logspace(3,5,base=10,num=10)
#alphas_ = np.arange(1000.,4000.,500).tolist()
alphas_ = np.arange(1000.,4000.,500).tolist()
model = lm.Lasso(fit_intercept = False)
for a in alphas_:
    print "alpha=%f"%a
    model.set_params(alpha=a)
    model.fit(X,y)
    print "training R2=%f"%model.score(X,y)
    print "validate R2=%f"%model.score(Xv,yv)
    print "test R2=%f"%model.score(Xt,yt)
```


## Regresión Lasso
```
...
alpha=1500.000000
training R2=0.999961
validate R2=0.536099
test R2=0.540964
alpha=2000.000000
training R2=0.999931
validate R2=0.543443
test R2=0.546293
alpha=2500.000000
training R2=0.999892
validate R2=0.548085
test R2=0.543148
...
```

## Conclusiones

Se probaron diferentes enfoques los que obtuvieron diferentes valores para el coeficiente de determinación. 

- Modelo de regresión lineal 
    - $R^2_{test} = 0.590$. Se intentó mejorar sobre este con distintos enfoques.

- Umbral de Varianza
    - Se redujo el número de características de 145265 a 58685
    - Mejora marginal de $R^2_{test} = R^2 = 0.594$. 
    - ¿Coincidencia?


## Conclusiones

Se probaron diferentes enfoques los que obtuvieron diferentes valores para el coeficiente de determinación. 

- Forward Stepwise Selection
    - Produjo en general malos resultados
    - Podría necesitar de otros criterios para aceptación de nuevas variables

- Lasso
    - No produjo una mejora respecto del modelo lineal
    - $R^2_{test} = 0.546 para $\alpha = 2000$
